# 05 Unexpected Flow


このノートブックは `config/config.example.yaml` を読み込み、synthetic データで手法を単体検証します。

In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "config" / "config.example.yaml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import load_config
from src.utils.calendar import month_end_dates, quarter_end_dates
from src.utils.io import save_parquet, save_csv, save_figure
from src.utils.validation import assert_weights_sum_to_one, assert_no_lookahead

cfg = load_config(PROJECT_ROOT / "config" / "config.example.yaml")
np.random.seed(cfg["project"]["seed"])

for rel in ["outputs/logs", "outputs/signals", "outputs/weights", "outputs/diagnostics", "outputs/figures"]:
    (PROJECT_ROOT / rel).mkdir(parents=True, exist_ok=True)

def make_synthetic(cfg):
    dates = pd.bdate_range(cfg["data"]["date_start"], cfg["data"]["date_end"])
    themes = cfg["data"]["assets"]["theme_ids"]
    n, m = len(dates), len(themes)

    factors = np.random.normal(0.0, 0.004, size=(n, 3))
    loadings = np.random.uniform(-1.0, 1.0, size=(m, 3))
    eps = np.random.normal(0.0, 0.01, size=(n, m))
    returns = pd.DataFrame(factors @ loadings.T + eps, index=dates, columns=themes)
    prices = 100.0 * (1.0 + returns).cumprod()

    flow_common = np.random.normal(0.0, 1.0, size=n)
    flow_noise = np.random.normal(0.0, 0.6, size=(n, m))
    flow = pd.DataFrame(flow_common[:, None] + flow_noise, index=dates, columns=themes)

    theme_scores = flow.rolling(21, min_periods=5).mean() + np.random.normal(0.0, 0.2, size=(n, m))
    stale_cols = themes[2::2]
    if len(stale_cols) > 0:
        theme_scores.loc[dates[-120:], stale_cols] = np.nan

    if cfg["calendar"]["rebalance"] == "Q":
        rebalance_dates = quarter_end_dates(dates)
    else:
        rebalance_dates = month_end_dates(dates)

    return dates, themes, returns, prices, flow, theme_scores, rebalance_dates


In [2]:
from src.signals.unexpected_flow import decompose_flow, predict_returns_with_unexpected_flow

dates, themes, returns, prices, flow, theme_scores, rebalance_dates = make_synthetic(cfg)

features_dict = {
    "ret_21": returns.rolling(21, min_periods=5).sum(),
    "vol_21": returns.rolling(21, min_periods=5).std(ddof=0),
    "mom_5": returns.rolling(5, min_periods=3).sum(),
}

pred_flow, unexpected_flow, info = decompose_flow(
    flow=flow,
    features_dict=features_dict,
    model=cfg["unexpected_flow"]["model"],
    standardize=cfg["unexpected_flow"]["standardize"],
)

report = predict_returns_with_unexpected_flow(
    returns=returns,
    unexpected_flow=unexpected_flow,
    horizon=cfg["unexpected_flow"]["horizon_days"],
)
report = report.join(info, how="left", rsuffix="_flow_model")

assert pred_flow.shape == flow.shape
assert unexpected_flow.shape == flow.shape

save_parquet(pred_flow, PROJECT_ROOT / "outputs/signals/05_pred_flow.parquet")
save_parquet(unexpected_flow, PROJECT_ROOT / "outputs/signals/05_unexpected_flow.parquet")
save_csv(report, PROJECT_ROOT / "outputs/diagnostics/05_unexpected_flow_report.csv")

print(report.tail())


                beta        ic  nobs        r2  nobs_flow_model  r2_flow_model
date                                                                          
2025-12-25 -0.005995 -0.485714     6  0.067517                6       0.087187
2025-12-26 -0.003641  0.257143     6  0.036404                6       0.239720
2025-12-29  0.005965  0.142857     6  0.018574                6       0.429073
2025-12-30 -0.000394  0.028571     6  0.000042                6       0.630612
2025-12-31       NaN       NaN     0       NaN                6       0.616028


In [3]:
required = [
    "outputs/signals/05_pred_flow.parquet",
    "outputs/signals/05_unexpected_flow.parquet",
    "outputs/diagnostics/05_unexpected_flow_report.csv",
]
status = "OK" if all((PROJECT_ROOT / p).exists() for p in required) else "NG"
checks = ", ".join([f"{p}:{(PROJECT_ROOT / p).exists()}" for p in required])
line = f"Notebook=05_unexpected_flow.ipynb, {status}, 生成ファイル確認={checks}\n"
with (PROJECT_ROOT / "outputs/logs/qa_checklist.txt").open("a", encoding="utf-8") as f:
    f.write(line)
print(line)


Notebook=05_unexpected_flow.ipynb, OK, 生成ファイル確認=outputs/signals/05_pred_flow.parquet:True, outputs/signals/05_unexpected_flow.parquet:True, outputs/diagnostics/05_unexpected_flow_report.csv:True

